In [48]:
import pandas as pd
import numpy as np
import main
import os
import fem
import math

In [2]:
path_to_folder = os.path.join(main.base_location, 'fem', 'processing', main.version)
df = fem.select_table_to_filelist(folder_path_to_list=path_to_folder, sheet_name=main.sheet_name)


Files in "/Users/ivanov.ev/Work/Dashboard/fem/processing/6+6" displayed below
--------------
1. processing_09092022.xlsx
--------------



In [ ]:
sheet_info_name = 'load'
file_name = 'ЦЭк.xlsx'
path_to_folder = os.path.join(main.base_location, 'fem', 'loading', main.version)

header = ['key']
years = [i for i in range(2015, 2051)]
header += years

final_df = None

In [ ]:
info = pd.read_excel(io=os.path.join(path_to_folder, file_name),
                   sheet_name=sheet_info_name
                   )

In [ ]:
for project_name ,sheet_name in zip(info.project, info.sheet):

    df = fem.extract_fem_from_excel_file(file_path=os.path.join(path_to_folder, file_name),
                                         sheet_name=sheet_name)
    new_header = list(set(df.columns) & set(header))

    df = df.loc[~df['key'].isnull(), new_header]